In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [48]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

training_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

training_data = training_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


In [52]:
input_size = 785
output_size = 10
hidden_layer = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer, activation='tanh'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])


In [53]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
NUM_EPOCHS = 5

model.fit(training_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), validation_steps=1, verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.2573 - accuracy: 0.9250 - val_loss: 0.1148 - val_accuracy: 0.9665
Epoch 2/5
540/540 - 3s - loss: 0.0990 - accuracy: 0.9697 - val_loss: 0.0748 - val_accuracy: 0.9788
Epoch 3/5
540/540 - 3s - loss: 0.0657 - accuracy: 0.9798 - val_loss: 0.0578 - val_accuracy: 0.9817
Epoch 4/5
540/540 - 3s - loss: 0.0493 - accuracy: 0.9849 - val_loss: 0.0543 - val_accuracy: 0.9837
Epoch 5/5
540/540 - 3s - loss: 0.0365 - accuracy: 0.9883 - val_loss: 0.0465 - val_accuracy: 0.9848


In [55]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 765ms/step - loss: 0.0781 - accuracy: 0.9774

In [56]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.74%
